<a href="https://colab.research.google.com/github/vieetdemort/Mysterious_Circles/blob/main/Mysterious_Circles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mysterious circles**

In [44]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
import seaborn as sns 
import matplotlib.pylab as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, roc_auc_score

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1n3_s-MAVuZCGsfqmU9RhoUYcnM_QU4o2NQvAmVfwgpc/edit?usp=sharing'

In the file dataset.csv you will find a synthetic selection. It contains 5 attributes and one target variable.

It is necessary to train a classifier with the highest generalization ability on this data. The metric used to evaluate the quality of the classifier is ROC-AUC. It should be noted in advance that if it is not possible to obtain a ROC-AUC above 70%, it means that some strong characters have not been invented. At the same time, it is suggested to pay attention not only to the accuracy of the classifier, but also to the design of the code, the study of the data, and a competent assessment of the quality of the resulting model - all this is also evaluated, and some of it can also help to create a better model.

The solution must be submitted in the form of a Python script or a Jupiter Notebook. Comments and diagrams are welcome.

In [ ]:
id = url.split('/')[5]

In [ ]:
df = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{id}/export?format=csv')

In [ ]:
df.head()


,feature_1,feature_2,feature_3,feature_4,feature_5,target
0,0.433306,-4.106981,1.252722,4.842634,7.610817,1
1,-0.216705,3.832332,-10.054348,-1.106758,1.987823,0
2,8.574821,-2.495187,5.404009,-6.219058,6.954602,1
3,7.673529,-4.234502,8.135008,3.748393,3.457908,0
4,3.548824,-2.588106,-2.095070,7.993792,-3.523852,1


In [ ]:
#checking for data types
df.dtypes.value_counts()

float64    5
int64      1
dtype: int64

In [13]:
#Checking data for a null value
df.isna().mean()

feature_1    0.0
feature_2    0.0
feature_3    0.0
feature_4    0.0
feature_5    0.0
target       0.0
dtype: float64

In [14]:
df['target'].mean()

0.6

## SPLIT DATA

In [20]:
X = df.drop(columns=['target']).copy()
Y = df['target']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## CATBOOST

In [37]:
model = CatBoostClassifier(verbose=100, eval_metric='AUC')

In [38]:
model.fit(X_train, y_train, eval_set=(X_test, y_test))

Learning rate set to 0.038062
0:	test: 0.5224025	best: 0.5224025 (0)	total: 4.88ms	remaining: 4.88s
100:	test: 0.5137499	best: 0.5434278 (2)	total: 240ms	remaining: 2.14s
200:	test: 0.5105113	best: 0.5434278 (2)	total: 478ms	remaining: 1.9s
300:	test: 0.5235173	best: 0.5434278 (2)	total: 725ms	remaining: 1.68s
400:	test: 0.5204743	best: 0.5434278 (2)	total: 969ms	remaining: 1.45s
500:	test: 0.5185332	best: 0.5434278 (2)	total: 1.21s	remaining: 1.21s
600:	test: 0.5193519	best: 0.5434278 (2)	total: 1.46s	remaining: 972ms
700:	test: 0.5199491	best: 0.5434278 (2)	total: 1.72s	remaining: 733ms
800:	test: 0.5232856	best: 0.5434278 (2)	total: 1.98s	remaining: 491ms
900:	test: 0.5228891	best: 0.5434278 (2)	total: 2.23s	remaining: 245ms
999:	test: 0.5237644	best: 0.5434278 (2)	total: 2.48s	remaining: 0us

bestTest = 0.54342778
bestIteration = 2

Shrink model to first 3 iterations.


In [56]:
#Calculate roc auc
roc_auc = roc_auc_score(y_true=y_test, y_score=model.predict_proba(X_test)[:,1])
roc_auc

0.5434277799803315